# 机器学习工程师纳米学位
## 监督学习
## 项目 2: 搭建一个学生干预系统

欢迎来到机器学习工程师纳米学位的第二个项目！在此文件中，有些示例代码已经提供给你，但你还需要实现更多的功能让项目成功运行。除非有明确要求，你无须修改任何已给出的代码。以**'练习'**开始的标题表示接下来的代码部分中有你必须要实现的功能。每一部分都会有详细的指导，需要实现的部分也会在注释中以**'TODO'**标出。请仔细阅读所有的提示！

除了实现代码外，你还**必须**回答一些与项目和你的实现有关的问题。每一个需要你回答的问题都会以**'问题 X'**为标题。请仔细阅读每个问题，并且在问题后的**'回答'**文字框中写出完整的答案。我们将根据你对问题的回答和撰写代码所实现的功能来对你提交的项目进行评分。

>**提示：**Code 和 Markdown 区域可通过 **Shift + Enter** 快捷键运行。此外，Markdown可以通过双击进入编辑模式。

### 问题 1 - 分类 vs. 回归
*在这个项目中你的任务是找出那些如果不给予帮助，最重可能无法毕业的学生。你觉得这个问题是哪种类型的监督学习问题，是分类问题还是回归问题？为什么？*

**答案: **这个问题属于监督学习，因为是否可以毕业是外界给的评判结论，并不是机器通过自己学习总结的规律及结论。另外，这个问题属于分类问题，原因是是否能够毕业是个标签值（而且是个布尔类型的标签值，值域可以用“是”或“否”表示），不是实数值。

## 分析数据
运行下面区域的代码以载入学生数据集，以及一些此项目所需的Python库。注意数据集的最后一列`'passed'`是我们的预测的目标（表示学生是毕业了还是没有毕业），其他的列是每个学生的属性。

In [7]:
# 载入所需要的库
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# 载入学生数据集
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### 练习: 分析数据
我们首先通过调查数据，以确定有多少学生的信息，并了解这些学生的毕业率。在下面的代码单元中，你需要完成如下的运算：
- 学生的总数， `n_students`。
- 每个学生的特征总数， `n_features`。
- 毕业的学生的数量， `n_passed`。
- 未毕业的学生的数量， `n_failed`。
- 班级的毕业率， `grad_rate`， 用百分数表示(%)。


In [8]:
# TODO： 计算学生的数量
n_students = student_data.index.size

# TODO： 计算特征数量
n_features = student_data.columns.size - 1

# TODO： 计算通过的学生数
n_passed = student_data.passed[student_data.passed == 'yes'].size

# TODO： 计算未通过的学生数
n_failed = student_data.passed[student_data.passed == 'no'].size

# TODO： 计算通过率
grad_rate = n_passed*100.0 / (n_passed + n_failed)

# 输出结果
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## 数据准备
在这个部分中，我们将要为建模、训练和测试准备数据
### 识别特征和目标列
你获取的数据中通常都会包含一些非数字的特征，这会导致一些问题，因为大多数的机器学习算法都会期望输入数字特征进行计算。

运行下面的代码单元将学生数据分成特征和目标列看一看他们中是否有非数字特征。

In [9]:
# 提取特征列
feature_cols = list(student_data.columns[:-1])

# 提取目标列 ‘passed’
target_col = student_data.columns[-1] 

# 显示列的列表
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# 将数据分割成特征数据和目标数据（即X_all 和 y_all）
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# 通过打印前5行显示特征信息
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### 预处理特征列

正如你所见，我们这里有几个非数值的列需要做一定的转换！它们中很多是简单的`yes`/`no`，比如`internet`。这些可以合理地转化为`1`/`0`（二元值，binary）值。

其他的列，如`Mjob`和`Fjob`，有两个以上的值，被称为_分类变量（categorical variables）_。处理这样的列的推荐方法是创建和可能值一样多的列（如：`Fjob_teacher`，`Fjob_other`，`Fjob_services`等），然后将其中一个的值设为`1`另外的设为`0`。

这些创建的列有时候叫做 _虚拟变量（dummy variables）_，我们将用[`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies)函数来完成这个转换。运行下面代码单元的代码来完成这里讨论的预处理步骤。

In [10]:
def preprocess_features(X):
    ''' 预处理学生数据，将非数字的二元特征转化成二元值（0或1），将分类的变量转换成虚拟变量
    '''
    
    # 初始化一个用于输出的DataFrame
    output = pd.DataFrame(index = X.index)

    # 查看数据的每一个特征列
    for col, col_data in X.iteritems():
        
        # 如果数据是非数字类型，将所有的yes/no替换成1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # 如果数据类型是类别的（categorical），将它转换成虚拟变量
        if col_data.dtype == object:
            # 例子: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # 收集转换后的列
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


In [11]:
X_all.head()

,school_GP,school_MS,sex_F,sex_M,age,address_R,address_U,famsize_GT3,famsize_LE3,Pstatus_A,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,1,0,1,0,18,0,1,1,0,1,...,1,0,0,4,3,4,1,1,3,6
1,1,0,1,0,17,0,1,1,0,0,...,1,1,0,5,3,3,1,1,3,4
2,1,0,1,0,15,0,1,0,1,0,...,1,1,0,4,3,2,2,3,3,10
3,1,0,1,0,15,0,1,1,0,0,...,1,1,1,3,2,2,1,1,5,2
4,1,0,1,0,16,0,1,1,0,0,...,1,0,0,4,3,2,1,2,5,4


### 实现: 将数据分成训练集和测试集
现在我们已经将所有的 _分类的（categorical）_ 特征转换成数值了。下一步我们将把数据（包括特征和对应的标签数据）分割成训练集和测试集。在下面的代码单元中，你需要完成下列功能：
- 随机混洗（shuffle）切分数据(`X_all`, `y_all`) 为训练子集和测试子集。
  - 使用300个数据点作为训练集（约75%），使用95个数据点作为测试集（约25%）。
  - 如果可能的话，为你使用的函数设置一个`random_state`。
  - 将结果存储在`X_train`, `X_test`, `y_train`和 `y_test`中。

In [12]:
# TODO：在这里导入你可能需要使用的另外的功能
from sklearn.model_selection import train_test_split

# TODO：设置训练集的数量
num_train = 300

# TODO：设置测试集的数量
num_test = X_all.shape[0] - num_train

# TODO：把数据集混洗和分割成上面定义的训练集和测试集
X_train = None
X_test = None
y_train = None
y_test = None

X_train, X_test, y_train, y_test = train_test_split(X_all,y_all,test_size=num_test*1.0/(num_train+num_test),random_state=20)
# 显示分割的结果
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## 训练和评价模型
在这个部分，你将选择3个适合这个问题并且在`scikit-learn`中已有的监督学习的模型。首先你需要说明你选择这三个模型的原因，包括这些数据集有哪些特点，每个模型的优点和缺点各是什么。然后，你需要将这些模型用不同大小的训练集（100个数据点，200个数据点，300个数据点）进行训练，并用F<sub>1</sub>的值来衡量。你需要制作三个表，每个表要显示训练集大小，训练时间，预测时间，训练集上的F<sub>1</sub>值和测试集上的F<sub>1</sub>值（每个模型一个表）。

**这是目前** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **里有的监督学习模型，你可以从中选择:**
- Gaussian Naive Bayes (GaussianNB) 朴素贝叶斯
- Decision Trees 决策树
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM) 向量模型机
- Logistic Regression 逻辑回归

### 问题 2 - 应用模型
*列出三个适合这个问题的监督学习算法模型。每一个你选择的模型：*

- 描述一个该模型在真实世界的一个应用场景。（你需要为此做点研究，并给出你的引用出处）
- 这个模型的优势是什么？他什么情况下表现最好？
- 这个模型的缺点是什么？什么条件下它表现很差？
- 根据我们当前数据集的特点，为什么这个模型适合这个问题。

**回答: **  
**逻辑回归**  
- 逻辑回归的应用场景之一是做广告是否点击预测。引用出处https://cloud.baidu.com/doc/BML/ScenarioTutorial.html  
- 逻辑回归属于线性算法的一种，具有模型简单、训练时间短、容易解释等优势。逻辑回归擅长分析线性关系，对于特征与结果之间呈现线性关系的场景表现较好，另外因为其训练与预测的速度都特别快，所以非常适合于大规模数据场景下使用。  
- 逻辑回归的缺点在于容易欠拟合，对极值比较敏感，分类精度可能不高。逻辑回归在非线性特征较多的场景下表现很差。  
- 当前数据集的特点是数据集规模小（只有395行），数据特征少（48个），数据特征既有标量特征又有实值特征，总体上是个简单的分类问题，可以使用逻辑回归来解决  
---
**集成方法（AdaBoost，Gradient Boosting）**  
- 集成方法被广泛应用到各种场景，用来提升单个模型的精度，应用例子：预测红酒的口感，引用出处Michael Bowles 的书《Machine Learning In Python》第六章https://www.amazon.com/Machine-Learning-Python-Techniques-Predictive/dp/1118961749/ref=sr_1_1?s=books&ie=UTF8&qid=1429853783&sr=1-1&keywords=machine+learning+in+python
- 集成方法的优势是泛化错误率低，精度高，不易过拟合。集成方法比较适用于行比列多很多的复杂问题（非线性模型）  
- 集成方法的缺点是对离群值比较敏感，对于列比行多的情况下，集成方法表现较差。 
- 当前数据集的特点是数据集规模小（只有395行），数据特征少（48个），总体上行比列多，虽然并不是复杂问题，但是集成方法同样可以处理
---
**SVM**  
- SVM被广泛应用于解决各种“分类”问题，应用例子：手写数字识别问题，引用出处Peter Harringto 的书《机器学习实战》第6章 https://www.amazon.cn/%E5%9B%BE%E4%B9%A6/dp/B00D747PTK
- SVM的优势在于可以解决高维问题，能够处理非线性特征的相互作用，无需依赖整个数据，泛化能力强。SVM特别适用于处理特征特别多的分类问题
- SVM的缺点在于对缺失数据敏感，对核函数的选择敏感，有时候很难找到一个合适的核函数。SVM不适合缺失数据太多的数据集，如果核函数选择不当，效果非常差
- 当前数据集的特点是数据集规模小（只有395行），数据特征少（48个），是个二元分类问题，没有缺失数据，SVM适用于解决这种分类问题


### 准备
运行下面的代码单元以初始化三个帮助函数，这三个函数将能够帮你训练和测试你上面所选择的三个监督学习算法。这些函数是：
- `train_classifier` - 输入一个分类器和训练集，用数据来训练这个分类器。
- `predict_labels` - 输入一个训练好的分类器、特征以及一个目标标签，这个函数将帮你做预测并给出F<sub>1</sub>的值.
- `train_predict` - 输入一个分类器以及训练集和测试集，它可以运行`train_clasifier`和`predict_labels`.
 - 这个函数将分别输出训练集的F<sub>1</sub>值和测试集的F<sub>1</sub>值

In [13]:
def train_classifier(clf, X_train, y_train):
    ''' 用训练集训练分类器 '''
    
    # 开始计时，训练分类器，然后停止计时
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)
    
    return "{:.4f}".format(end-start)

    
def predict_labels(clf, features, target):
    ''' 用训练好的分类器做预测并输出F1值'''
    
    # 开始计时，作出预测，然后停止计时
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # 输出并返回结果
    run_time = "{:.4f}".format(end - start)
    score = "{:.4f}".format(f1_score(target.values, y_pred, pos_label='yes'))
    print "Made predictions in {} seconds.".format(run_time)
    return (run_time,score)


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' 用一个分类器训练和预测，并输出F1值 '''
    
    li = []
    # 输出分类器名称和训练集大小
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    li.append(len(X_train))
    # 训练一个分类器
    run_time = train_classifier(clf, X_train, y_train)
    li.append(run_time)
    
    pred_time_train,score_train = predict_labels(clf, X_train, y_train)
    pred_time_test,score_test = predict_labels(clf, X_test, y_test)
    
    li.append(pred_time_test)
    li.append(score_train)
    li.append(score_test)
    # 输出训练和测试的预测结果
    print "F1 score for training set: {}.".format(score_train)
    print "F1 score for test set: {}.".format(score_test)
    
    return li

### 练习: 模型评价指标
借助于上面定义的函数，你现在需要导入三个你选择的监督学习模型，然后为每一个模型运行`train_predict`函数。请记住，对于每一个模型你需要在不同大小的训练集（100，200和300）上进行训练和测试。所以，你在下面应该会有9个不同的输出（每个模型都有训练集大小不同的三个输出）。在接下来的代码单元中，你将需要实现以下功能：
- 引入三个你在上面讨论过的监督式学习算法模型。
- 初始化三个模型并将它们存储在`clf_A`， `clf_B` 和 `clf_C`中。
 - 如果可能对每一个模型都设置一个`random_state`。
 - **注意:** 这里先使用每一个模型的默认参数，在接下来的部分中你将需要对某一个模型的参数进行调整。
- 创建不同大小的训练集用来训练每一个模型。
 - *不要再混洗和再分割数据！新的训练集要取自`X_train`和`y_train`.*
- 对于每一个模型要用不同大小的训练集来训练它，然后在测试集上做测试（总共需要9次训练测试）   
**注意:** 在下面的代码单元后面我们提供了三个表用来存储你的结果。

In [14]:
# TODO：从sklearn中引入三个监督学习模型
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import LinearSVC

# TODO：初始化三个模型

clf_A = LogisticRegression(random_state = 0)
clf_B = AdaBoostClassifier(random_state = 0)
clf_C = GradientBoostingClassifier(random_state = 0)
clf_D = LinearSVC(random_state = 0)


# TODO：设置训练集大小
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train
y_train_300 = y_train

# TODO：对每一个分类器和每一个训练集大小运行'train_predict' 
# train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
# train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
# train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)

# train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
# train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
# train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)

# train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
# train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
# train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

cols = ['训练集大小','训练时间','预测时间 (测试)','F1值 (训练)','F1值 (测试)']

def model_train_predict(model):
    
    df = pd.DataFrame(columns=cols)

    li_100 = train_predict(model, X_train_100, y_train_100, X_test, y_test)
    li_200 = train_predict(model, X_train_200, y_train_200, X_test, y_test)
    li_300 = train_predict(model, X_train_300, y_train_300, X_test, y_test)

    df = df.append(pd.DataFrame([li_100],columns=cols),ignore_index=True)
    df = df.append(pd.DataFrame([li_200],columns=cols),ignore_index=True)
    df = df.append(pd.DataFrame([li_300],columns=cols),ignore_index=True)
    
    return df

df_A = model_train_predict(clf_A)
df_B = model_train_predict(clf_B)
df_C = model_train_predict(clf_C)
df_D = model_train_predict(clf_D)

Training a LogisticRegression using a training set size of 100. . .
Trained model in 0.8704 seconds
Made predictions in 0.1246 seconds.
Made predictions in 0.0004 seconds.
F1 score for training set: 0.9536.
F1 score for test set: 0.7338.
Training a LogisticRegression using a training set size of 200. . .
Trained model in 0.0050 seconds
Made predictions in 0.0379 seconds.
Made predictions in 0.0004 seconds.
F1 score for training set: 0.8707.
F1 score for test set: 0.7586.
Training a LogisticRegression using a training set size of 300. . .
Trained model in 0.0065 seconds
Made predictions in 0.0004 seconds.
Made predictions in 0.0003 seconds.
F1 score for training set: 0.8311.
F1 score for test set: 0.8000.
Training a AdaBoostClassifier using a training set size of 100. . .
Trained model in 0.1538 seconds
Made predictions in 0.0075 seconds.
Made predictions in 0.0056 seconds.
F1 score for training set: 0.9930.
F1 score for test set: 0.7328.
Training a AdaBoostClassifier using a training s

In [80]:
# from sklearn.ensemble import GradientBoostingClassifier

# clf_E = GradientBoostingClassifier(random_state = 0)

# cols = ['训练集大小','训练时间','预测时间 (测试)','F1值 (训练)','F1值 (测试)']

# df = pd.DataFrame(columns=cols)

# li_100 = train_predict(clf_E, X_train_100, y_train_100, X_test, y_test)
# li_200 = train_predict(clf_E, X_train_200, y_train_200, X_test, y_test)
# li_300 = train_predict(clf_E, X_train_300, y_train_300, X_test, y_test)

# df = df.append(pd.DataFrame([li_100],columns=cols),ignore_index=True)
# df = df.append(pd.DataFrame([li_200],columns=cols),ignore_index=True)
# df = df.append(pd.DataFrame([li_300],columns=cols),ignore_index=True)
# df

### 结果表格
编辑下面的表格看看在[Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables)中如何设计一个表格。你需要把上面的结果记录在表格中。

** 分类器 1 - 逻辑回归** 

In [15]:
df_A

,训练集大小,训练时间,预测时间 (测试),F1值 (训练),F1值 (测试)
0,100.0,0.8704,0.0004,0.9536,0.7338
1,200.0,0.0050,0.0004,0.8707,0.7586
2,300.0,0.0065,0.0003,0.8311,0.8000


** 分类器 2 - 集成方法（AdaBoost）**

In [16]:
df_B

,训练集大小,训练时间,预测时间 (测试),F1值 (训练),F1值 (测试)
0,100.0,0.1538,0.0056,0.9930,0.7328
1,200.0,0.1304,0.0042,0.8966,0.7500
2,300.0,0.1123,0.0035,0.8396,0.8054


** 分类器 3 - 集成方法（GradientBoost）**

In [17]:
df_C

,训练集大小,训练时间,预测时间 (测试),F1值 (训练),F1值 (测试)
0,100.0,0.0375,0.0003,1.0000,0.7612
1,200.0,0.0474,0.0003,0.9858,0.8299
2,300.0,0.0595,0.0003,0.9592,0.8108


** 分类器 4 - SVM（向量模型机）**

In [18]:
df_D

,训练集大小,训练时间,预测时间 (测试),F1值 (训练),F1值 (测试)
0,100.0,0.0055,0.0001,0.9730,0.6341
1,200.0,0.0126,0.0001,0.8814,0.7586
2,300.0,0.0243,0.0001,0.8204,0.7973


## 选择最佳模型
在最后这一部分中，你将从三个监督学习模型中选择一个用在学生数据上的最佳模型。然后你将在最佳模型上用全部的训练集（`X_train`和`y_train`）运行一个网格搜索算法，在这个过程中，你要至少调整一个参数以提高模型的F<sub>1</sub>值（相比于没有调参的模型的分值有所提高）。 

### 问题 3 - 选择最佳模型
*给予你上面做的实验，用一到两段话，向（学校）监事会解释你将选择哪个模型作为最佳的模型。哪个模型在现有的数据，有限的资源、开支和模型表现综合来看是最好的选择？*

**回答: **  

我会选择集成方法(GradientBoost)作为最佳模型.原因有以下几点：
- 从F1值来看，无论是训练集还是测试集，集成方法(GradientBoost)都具有最佳表现
- 从F1值来看，无论是数据集规模大小，集成方法(GradientBoost)都具有最佳表现
- 由于数据集整体规模较小，训练时间虽有差异，但是不成为关键要素，可以忽略


### 问题 4 - 用通俗的语言解释模型
*用一到两段话，向（学校）监事会用外行也听得懂的话来解释最终模型是如何工作的。你需要解释所选模型的主要特点。例如，这个模型是怎样被训练的，它又是如何做出预测的。避免使用高级的数学或技术术语，不要使用公式或特定的算法名词。*

**回答: **  
集成方法(GradientBoost)是基于决策树的集成方法.决策树就是一个判断逻辑的集合,即如何符合XX条件,则结果是YY.  
训练-预测过程大概是这样的:  
- 首先根据已有数据训练出来一个简单的决策树模型,然后用模型做预测,将预测结果与实际结果做比较,找出其中差异，得到预测不准的部分
- 对于预测不准的部分赋予更高的权重,然后再训练一个决策树模型，在训练中尽量向减少预测误差的方向优化,然后做预测，找差值，得到不准的部分
- 继续循环上一步的训练-预测动作，直到达到模型数量阈值或者某个准确度
- 将上述所有模型集成起来,做预测时,参考所有模型的权重和给出预测结果,加总汇集之后给出最终的预测结果

### 练习: 模型调参
细调选择的模型的参数。使用网格搜索（`GridSearchCV`）来至少调整模型的重要参数（至少调整一个），这个参数至少需给出并尝试3个不同的值。你要使用整个训练集来完成这个过程。在接下来的代码单元中，你需要实现以下功能：
- 导入 [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) 和 [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- 创建一个对于这个模型你希望调整参数的字典。
 - 例如: `parameters = {'parameter' : [list of values]}`。
- 初始化你选择的分类器，并将其存储在`clf`中。
- 使用`make_scorer` 创建F<sub>1</sub>评分函数并将其存储在`f1_scorer`中。
 - 需正确设定参数`pos_label`的值！
- 在分类器`clf`上用`f1_scorer` 作为评价函数运行网格搜索,并将结果存储在`grid_obj`中。
- 用训练集(`X_train`, `y_train`)训练grid search object,并将结果存储在`grid_obj`中。

In [19]:
# TODO: 导入 'GridSearchCV' 和 'make_scorer'
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
# TODO：创建你希望调整的参数列表
# 适用于逻辑回归算法的参数
# parameters = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }

parameters = {'max_depth':range(1,11)}
# TODO：初始化分类器
clf = GradientBoostingClassifier()

# TODO：用'make_scorer'创建一个f1评分函数
f1_scorer = make_scorer(f1_score,pos_label='yes')

# TODO：在分类器上使用f1_scorer作为评分函数运行网格搜索
grid_obj = GridSearchCV(clf,parameters,f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
# TODO：用训练集训练grid search object来寻找最佳参数
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
# 得到预测的结果
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
# 输出经过调参之后的训练集和测试集的F1值
pred_time_train,score_train = predict_labels(clf, X_train, y_train)
pred_time_test,score_test = predict_labels(clf, X_test, y_test)
print "Tuned model has a training F1 score of {}.".format(score_train)
print "Tuned model has a testing F1 score of {}.".format(score_test)

Made predictions in 0.0003 seconds.
Made predictions in 0.0004 seconds.
Tuned model has a training F1 score of 0.8356.
Tuned model has a testing F1 score of 0.8366.


### 问题 5 - 最终的 F<sub>1</sub> 值
*最终模型的训练和测试的F<sub>1</sub>值是多少？这个值相比于没有调整过参数的模型怎么样？*

**回答: **  
最终模型训练的F1值为0.8356,测试的F1值为0.8366.相比于没有调参的模型(只跟训练集为300的数据对比),训练时的F1值由0.9592降低为0.8356，测试时3的F1值由0.8108提升为0.8366，训练和测试的F1值差值缩小，其结果是通过调参，修正了过拟合，增强了模型的泛化能力，进而提升了测试时的最终指标。

> **注意**: 当你写完了所有的代码，并且回答了所有的问题。你就可以把你的 iPython Notebook 导出成 HTML 文件。你可以在菜单栏，这样导出**File -> Download as -> HTML (.html)**把这个 HTML 和这个 iPython notebook 一起做为你的作业提交。  